# Readme


1.   Run all the cells in order.
2.   Torch versions are specified to suppress warnings
3.   Git directory cloned to have the dataset available for each runtime
4.   DarkCovidNet model is sourced from the same github repository
5.   DenseNet121, DenseNet169, ResNet34 are all implemented in Fastai




In [ ]:
!pip install "torch==1.4" "torchvision==0.5.0"

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import fastai
from fastai.vision import *

In [ ]:
!git clone https://github.com/muhammedtalo/COVID-19.git

### Data

In [ ]:
path = Path('/content/COVID-19/X-Ray Image DataSet')

In [ ]:
np.random.seed(41)
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2,
        ds_tfms=get_transforms(), size=(256,256), bs=32, num_workers=4).normalize()

In [ ]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

In [ ]:
print("Number of examples in training:", len(data.train_ds))
print("Number of examples in validation:", len(data.valid_ds))

In [ ]:
xb,yb = data.one_batch()
xb.shape,yb.shape

In [ ]:
#Sample images from the dataset
data.show_batch(rows=3, figsize=(10,10))

## DarkCovidNet (modified Darknet model)

In [ ]:
def conv_block(ni, nf, size=3, stride=1):
    for_pad = lambda s: s if s > 2 else 3
    return nn.Sequential(
        nn.Conv2d(ni, nf, kernel_size=size, stride=stride,
                  padding=(for_pad(size) - 1)//2, bias=False), 
        nn.BatchNorm2d(nf),
        nn.LeakyReLU(negative_slope=0.1, inplace=True)  
    )

In [ ]:
def triple_conv(ni, nf):
    return nn.Sequential(
        conv_block(ni, nf),
        conv_block(nf, ni, size=1),  
        conv_block(ni, nf)
    )

In [ ]:
def maxpooling():
    return nn.MaxPool2d(2, stride=2)

In [ ]:
model = nn.Sequential(
    conv_block(3, 8),
    maxpooling(),
    conv_block(8, 16),
    maxpooling(),
    triple_conv(16, 32),
    maxpooling(),
    triple_conv(32, 64),
    maxpooling(),
    triple_conv(64, 128),
    maxpooling(),
    triple_conv(128, 256),
    conv_block(256, 128, size=1),
    conv_block(128, 256),
    conv_layer(256, 2),
    Flatten(),
    nn.Linear(338, 2)
)

In [ ]:
learn = Learner(data, model, loss_func = nn.CrossEntropyLoss(), metrics=accuracy)

In [ ]:
print(learn.summary())

In [ ]:
learn.fit_one_cycle(100, max_lr=3e-3)

In [ ]:
print("Number of examples in testing:", len(data.valid_ds))

In [ ]:
probs,targets = learn.get_preds(ds_type=DatasetType.Valid) 

In [ ]:
accuracy(probs,targets)

In [ ]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

In [ ]:
probs = np.argmax(probs, axis=1)
correct = 0
for idx, pred in enumerate(probs):
    if pred == targets[idx]:
        correct += 1
accuracy = correct / len(probs)
print(len(probs), correct, accuracy)

from sklearn.metrics import confusion_matrix
np.set_printoptions(threshold=np.inf) 
cm1 = confusion_matrix(targets, probs)
print(cm1)

from sklearn.metrics import classification_report
y_true1 = targets
y_pred1 = probs
target_names = ['Covid-19', 'No_findings']
print(classification_report(y_true1, y_pred1, target_names=target_names))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
cm_fig = interp.plot_confusion_matrix(return_fig=True)
ax = cm_fig.gca()
ax.set_ylim(interp.data.c - .5, - .5);

# DENSENET121

In [ ]:
learn_dense = cnn_learner(data, models.densenet121, metrics = error_rate)

In [ ]:
print(learn_dense.summary())

In [ ]:
learn_dense.fit(20)

In [ ]:
learn_dense.recorder.plot_losses()

In [ ]:
probs,targets = learn_dense.get_preds(ds_type=DatasetType.Valid) 

In [ ]:
accuracy(probs,targets)

In [ ]:
probs = np.argmax(probs, axis=1)
correct = 0
for idx, pred in enumerate(probs):
    if pred == targets[idx]:
        correct += 1
accuracy = correct / len(probs)
print(len(probs), correct, accuracy)

from sklearn.metrics import confusion_matrix
np.set_printoptions(threshold=np.inf) # shows whole confusion matrix
cm1 = confusion_matrix(targets, probs)
print(cm1)

from sklearn.metrics import classification_report
y_true1 = targets
y_pred1 = probs
target_names = ['Covid-19', 'No_findings', 'Pneumonia']
print(classification_report(y_true1, y_pred1, target_names=target_names))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn_dense)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
cm_fig = interp.plot_confusion_matrix(return_fig=True)
ax = cm_fig.gca()
ax.set_ylim(interp.data.c - .5, - .5);

# DENSENET169

In [ ]:
learn_dense169 = cnn_learner(data, models.densenet169, metrics = error_rate)

In [ ]:
print(learn_dense169.summary())

In [ ]:
learn_dense169.fit(20)

In [ ]:
learn_dense169.recorder.plot_losses()

In [ ]:
probs,targets = learn_dense169.get_preds(ds_type=DatasetType.Valid) 

In [ ]:
accuracy(probs,targets)

In [ ]:
probs = np.argmax(probs, axis=1)
correct = 0
for idx, pred in enumerate(probs):
    if pred == targets[idx]:
        correct += 1
accuracy = correct / len(probs)
print(len(probs), correct, accuracy)

from sklearn.metrics import confusion_matrix
np.set_printoptions(threshold=np.inf) # shows whole confusion matrix
cm1 = confusion_matrix(targets, probs)
print(cm1)

from sklearn.metrics import classification_report
y_true1 = targets
y_pred1 = probs
target_names = ['Covid-19', 'No_findings', 'Pneumonia']
print(classification_report(y_true1, y_pred1, target_names=target_names))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn_dense169)

In [ ]:
cm_fig = interp.plot_confusion_matrix(return_fig=True)
ax = cm_fig.gca()
ax.set_ylim(interp.data.c - .5, - .5);

# RESNET34

In [ ]:
learn_res34 = cnn_learner(data, models.resnet34, metrics = error_rate)

In [ ]:
print(learn_res34.summary())

In [ ]:
learn_res34.fit(20)

In [ ]:
learn_res34.recorder.plot_losses()

In [ ]:
probs,targets = learn_res34.get_preds(ds_type=DatasetType.Valid) 

In [ ]:
probs = np.argmax(probs, axis=1)
correct = 0
for idx, pred in enumerate(probs):
    if pred == targets[idx]:
        correct += 1
accuracy = correct / len(probs)
print(len(probs), correct, accuracy)

from sklearn.metrics import confusion_matrix
np.set_printoptions(threshold=np.inf) # shows whole confusion matrix
cm1 = confusion_matrix(targets, probs)
print(cm1)

from sklearn.metrics import classification_report
y_true1 = targets
y_pred1 = probs
target_names = ['Covid-19', 'No_findings', 'Pneumonia']
print(classification_report(y_true1, y_pred1, target_names=target_names))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn_res34)

In [ ]:
cm_fig = interp.plot_confusion_matrix(return_fig=True)
ax = cm_fig.gca()
ax.set_ylim(interp.data.c - .5, - .5);